いまさらながら、このコンペ「M5 Forecasting Challenge」に参加することにしたので、とりあえず与えられたデータがどんなものなのか、を正確に理解しておこうと思い、このノートブックに記載しました。

個人的な備忘録ですが、せっかくなので公開しようと思います。

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df_calendar = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
df_sales_train_evaluation = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv')
df_sales_train_validation = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')
df_sell_prices = pd.read_csv('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')

# ■ calendar.csv の内容を確認

まずはinfoメソッドで要約情報を確認します。

In [ ]:
df_calendar.info()

## date
dateは、YYYY-MM-DD のフォーマットの日付情報のようです。このデータファイルにおける主キーとなっていると考えられます。

## wm_yr_wk
wm_yr_wkは、'1' + 西暦下2桁 + '週数2桁' というフォーマットになっているようです。

たとえば、2015年の第5週目だと、'11505' となるようです。（先頭の'1'の意味って何だろう？とりあえず気にしなくていいか・・・）

気を付けるべき点があるとすれば、西暦下2桁は西暦そのままではなく、各年の1月の最終土曜日から新しい年度になっているということでしょうか。

## weekday, wday
weekdayとwdayは以下の表のように対応しているようです。

土曜日始まりなのはなぜなのだろう？

|weekday|wday|
|:-----:|:--:|
|Saturday|1|
|Sunday|2|
|Monday|3|
|Tuesday|4|
|Wednesday|5|
|Thursday|6|
|Friday|7|

## month, year

何月か、西暦何年か、がそのまま入っているようです。

## d
2011年1月29日を d_1 として、そこから連番となっているようです。

## event_type, event_name

event_typeはイベントのカテゴリのようなもので、event_nameは個別のイベント名のようです。

それぞれ 1 と 2 がありますが、意味的な違いがあるわけではなく、イベントが重複する日には 2 を使うようです。

ですので、分析の際には One Hot Encoding などの前処理が必要そうに思われます。

In [ ]:
df_calendar.groupby(['event_type_1', 'event_name_1'])['date'].count()

In [ ]:
df_calendar.groupby(['event_type_2', 'event_name_2'])['date'].count()

## snap_CA, snap_TX, snap_WI

これは初見では意味不明な項目だったのですが、https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/135883 を読んで理解しました。

SNAPというのは、低所得者向けの「補助的栄養支援プログラム」(Supplemental Nutrition Assistance Program, SNAP) のことで、毎月「SNAPの日」がいくつかあり、受給者はそのうちのどれかの日に食料品購入を目的とした支給を受けるようです。で、カルフォルニア州の「SNAPの日」がsnap_CA、テキサス州の「SNAPの日」がsnap_TX、ウィスコンシン州の「SNAPの日」がsnap_WI、ということだそうです。

食料品購入用の支給は、別に「SNAPの日」に使い切らなくても良いらしいのですが、支給対象となるのが低所得者なので、「SNAPの日」からそんなに日数を経ずして使い切ってしまうことが多いようです。

なので、「SNAPの日」の直後、という条件が食料品の購入を底上げする可能性はありそうです。

# ■ sales_train_validation(evaluation).csv の内容を確認

sales_train_validationに比べるとsales_train_evaluationは、売上数量が入っている日数に違いがありますが、それ以外のフォーマットは同じなので、併せて概観していきます。

## id

これはどうも item_id の末尾に、validaition か evaluation のいずれかをつけたもののようです。

最終的な予測では、[item_id]_validation ならば、該当の商品・店舗の d_1913 から d_1941 の売上を予測し、 [item_id]_evaluation ならば、 d_1942 から d_1960 の売上を予測することになるようです。

同じ商品・店舗の組み合わせで、異なる期間で予測をするため、idを分けている、ということなのですかね。

ちなみに、Public Leaderboard の正解値はすべてsales_train_evaluationに含まれるそうなので、Public Leaderboardは全く参考にならないようです。

## item_id, dept_id, cat_id

item_idは特定の商品、dept_idはその商品が属する分類で、cat_idはさらにそのdept_idが属する大分類とでもいったものだと思われます。

たとえば、コカ・コーラ ∈ 炭酸飲料 ∈ 飲料 みたいな包含関係のある、商品―商品分類関係ですかね。

ちなみに、これらはマスク処理が施されているのか、具体的な商品名はわからないようになっています。きっとリークを防ぐためでしょうね。

以下を見てわかる通り、対象となる商品は、食料品、ホビー、住居品の3種類のようです。

In [ ]:
df_sales_train_validation.groupby(['cat_id', 'dept_id'])['item_id'].count()

## state_id, store_id

その名の通り、どの州のどの店舗での売上か、を意味するものと思われます。

以下を見てわかる通り、対象となる店舗は全10店舗あり、3つの州にまたがっています。


In [ ]:
df_sales_train_validation.groupby(['state_id', 'store_id'])['item_id'].count()

## d_*N*

2011年1月29日を1日目として、*N*日目の売上数量を意味しているようです。

# ■ sell_prices.csv の内容を確認

このファイルには店別、商品別、週別の商品売価が記載されています。

ウォルマートは週単位で売価をコントロールしているのですね・・・

注意するポイントがあるとすると、販売開始前、販売終了後、店舗取り扱いなし、といった場合、売価が存在していない可能性があるということでしょうか。（実際に販売前商品には売価が設定されていないようです）

ある店では取り扱いをやめて売価設定がないはずだけど、しばらく遅れて、ぽっと売れた、みたいなパターンも探せばあるかもしれません。そういうケースがもしあったら、例外ではじいてしまってもいいかもしれませんね。

・・・まあ、ここではそんなケースが存在するかまでは、まだ見ていませんが。。。